In [ ]:
# %pip install gradio

In [ ]:
# sample 코드 돌려보기
import gradio as gr
def update(name):
    return f"Welcome to Gradio, {name}!"

with gr.Blocks() as demo:
    gr.Markdown("Start typing below and then click **Run** to see the output.")
    with gr.Row():
        inp = gr.Textbox(placeholder="What is your name?")
        out2 = gr.Textbox()
    btn = gr.Button("Run")
    btn.click(fn=update, inputs=inp, outputs=out2)

demo.launch()
# demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


### dotenv 관련

In [12]:
import os
import requests
import time
import platform
import random
from PIL import Image, ImageDraw, ImageFont
from dotenv import load_dotenv

load_dotenv()

api_endpoint = os.getenv("DOCUMENT_INTELLIGENCE_ENDPOINT")
api_key = os.getenv("DOCUMENT_INTELLIGENCE_API_KEY")

### jupyter 환경 에서 gradio 돌릴때 eventloop 중첩 안되게 하는 설정

In [2]:
## jupyter 에서 gradio 돌릴때 eventloop 중첩 안되게 하는 설정
import nest_asyncio
nest_asyncio.apply()

## GRADIO 화면 만들기

In [ ]:
import gradio as gr

# gradio ui 구성
with gr.Blocks() as demo :

    def change_img(img) :
        print(img)
        return img

    gr.Markdown("# Azure Document Intelligence")

    with gr.Row() :
        input_img = gr.Image(label="이미지 선택", type="filepath", width=500)
        # output_img = gr.Image(label="결과 이미지", type="pil", interactive=False, width=500)
        # input_img = gr.Image(label="이미지 선택")
        output_img = gr.Image(label="결과 이미지")

    input_img.change(fn=change_img, inputs=[input_img], outputs=[output_img])

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\fastapi\applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\sit

current status : running
current status : succeeded


Traceback (most recent call last):
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EL91\AppData\Local\Progra

In [14]:
def request_document_intelligence(img_pth) :

    # 1. Endpoint
    endpoint = api_endpoint
    
    # 2. METHOD : POST

    # 3. Header
    headers = {
        "Content-Type" : "image/*",
        "Ocp-Apim-Subscription-Key" : api_key
    }

    # 4. Body

    with open(img_pth, "rb") as img_file :
        img_data = img_file.read()

    body = img_data

    response = requests.post(endpoint, headers=headers, data=body)

    if response.status_code != 202 :
        print("Error : ", response.status_code, response.text)
        return None
    
    url = response.headers['Operation-Location']


    while True :
        result_response=requests.get(url, headers=headers)

        if result_response.status_code != 200 :
            print("Error : ", result_response.status_code, result_response.text)
            return None
        
        result_json = result_response.json()
        current_status = result_json.get('status')

        if current_status == 'running' :
            print('current status :', current_status)
            time.sleep(1)
            continue

        else :
            break

    print('current status :', current_status)

    if current_status != 'succeeded' :
        return None

    return result_response.json()

def random_color():
    """
    시각화를 위해 랜덤한 RGB 색상 튜플 (R, G, B)을 반환합니다.
    """
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def get_font():
    """
    운영체제(OS)에 따라 한글이 지원되는 적절한 폰트 객체를 반환합니다.
    이미지에 텍스트를 그릴 때 깨짐을 방지하기 위함입니다.
    """
    font_size = 20
    
    try:
        if platform.system() == "Windows":
            # 윈도우: 맑은 고딕
            return ImageFont.truetype("malgun.ttf", font_size)
        elif platform.system() == "Darwin":  # macOS
            # 맥: 애플 고딕
            return ImageFont.truetype("AppleGothic.ttf", font_size)
        else:  # Linux 등
            # 기본 폰트 (한글 지원 안 될 수 있음)
            return ImageFont.load_default(size=font_size)
    except IOError:
        # 지정한 폰트 파일이 없을 경우 PIL 기본 폰트 사용
        return ImageFont.load_default()

def draw_image(img_pth, response_data) :
    img = Image.open(img_pth)
    draw = ImageDraw.Draw(img)

    block_list = response_data.get("analyzeResult").get("paragraphs")

    for block in block_list :
        color = random_color()
        content = block.get("content")
        polygon = block.get("boundingRegions")[0].get("polygon")
        pairs = list(zip(polygon[::2], polygon[1::2]))
        draw.polygon(pairs, outline=color)
        draw.text((pairs[0][0], pairs[0][1] - 20), content, fill=color, font=get_font())

    return img
    
import gradio as gr

# gradio ui 구성
with gr.Blocks() as demo :


    def change_img(img_pth) :
        response_data = request_document_intelligence(img_pth)
        img = draw_image(img_pth, response_data)
        return img

    gr.Markdown("# Azure Document Intelligence")

    with gr.Row() :
        input_img = gr.Image(label="이미지 선택", type="filepath", width=500)
        output_img = gr.Image(label="결과 이미지", type="pil", interactive=False, width=500)

    input_img.change(fn=change_img, inputs=[input_img], outputs=[output_img])

demo.launch()

# 둘리 신분증
# request_document_intelligence(r"C:/workspace/azure-cloud-deeplearning-exer/document-intelligence/example-idcard.jpg")

# 문서 예시
# request_document_intelligence(r"C:/workspace/azure-cloud-deeplearning-exer/document-intelligence/example-document.png")

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\fastapi\applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\sit

current status : succeeded
